### From first part of Assignment

In [6]:
!pip install watson-developer-cloud==1.5

In [7]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

### Fetch Data

In [8]:
urlx="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_content = requests.get(urlx).text

# Parse the html content
soup = BeautifulSoup(html_content, "lxml")
#get the table
ziptable = soup.find("table", attrs={"class": "wikitable"})
# get the rows in the body
zipdata = ziptable.tbody.find_all("tr") 
# Get all the headings of Lists
headings = []
for td in zipdata[0].find_all("th"):
    # remove any newlines and extra spaces from left and right
    headings.append(td.text.replace('\n', ' ').strip())

#prepare data frame    
df = pd.DataFrame(columns = headings, index= range(0,len(zipdata)-1))

#collect table data into datafram
row_marker = 0
for row in zipdata:
    column_marker = 0
    columns = row.find_all("td")
    for column in columns:
        df.iat[row_marker,column_marker] = column.get_text()
        column_marker += 1
    if len(columns) >0:
        row_marker += 1
#check data frame

#apply Borough requirement
df2 = df[~df['Borough'].str.contains('Not assigned', na=False)]

#apply Neighbourhood requirement                 
df2['Neighbourhood'] = np.where(df2['Neighbourhood'].str.contains('Not assigned', na=False),  df2['Borough'], df2['Neighbourhood'])
df2.head(10)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n
7,M6A,North York,Lawrence Manor\n
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue\n
11,M1B,Scarborough,Rouge\n
12,M1B,Scarborough,Malvern\n


In [9]:
## GET Geocoder package

In [10]:
url = "http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv"
dfcord = pd.read_csv(url)
dfcord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Change Column Name to be common

In [17]:
df2 = df2.rename(columns={'Postcode': 'Postalcode'})
dfcord = dfcord.rename(columns={'Postal Code': 'Postalcode'})                       
display(df2.head())
display(dfcord.head())

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n


,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge tables

In [18]:
df3 = df2.merge(dfcord, on='Postalcode', how='left')
df3.head(10)

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods\n,43.753259,-79.329656
1,M4A,North York,Victoria Village\n,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront\n,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park\n,43.654260,-79.360636
4,M6A,North York,Lawrence Heights\n,43.718518,-79.464763
5,M6A,North York,Lawrence Manor\n,43.718518,-79.464763
6,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
7,M9A,Etobicoke,Islington Avenue\n,43.667856,-79.532242
8,M1B,Scarborough,Rouge\n,43.806686,-79.194353
9,M1B,Scarborough,Malvern\n,43.806686,-79.194353
